In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import geopandas
import os
import requests

In [ ]:
## specify areas of interest / districts
## URL method accesses an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("shapeName") to create properties for output file
#URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-CHN-Ningbo-ADM1.geojson'
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/cities4forests/data/boundaries/MEX-Mexico_City-ADM2.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
DistrictsProjCRS = Districts.geometry().projection().crs()

print(DistrictsProjCRS.getInfo())
print(Districts.first().toDictionary().getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]
#ADMlevel = basename.split('-')[-1]

Areaofinterest = AOIname ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
#unitofanalysis = ADMlevel ## options: "ADM0" (country), "ADM1" (state), "Metro" (metropolitan region), "ADM2" (municipality), "ADM3" (subcity/locality), "ADM4"(ward/neighborhood), ideally align with https://www.geoboundaries.org/index.html#getdata

print(Areaofinterest)
#print(unitofanalysis)

In [ ]:
## create map
Map = geemap.Map(height="350px")

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=10)

In [ ]:
Map.addLayer(Districts,{},"Districts")
Map

In [ ]:
pop = ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj')
pop = (pop.filter(ee.Filter.And(
    ee.Filter.bounds(Districts),
    ee.Filter.inList('year',[2020])))
    .select('population'))
popImg = pop.mean().rename('population')

popScale = pop.first().projection().nominalScale()

popviz = {
  'min': 0.0,
  'max': 150.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
};
       
Map.addLayer(popImg,popviz,"Population")

In [ ]:
# get bounding box for Districts area

bb = Districts.union(1).geometry().buffer(100).bounds(1000)
print(bb.getInfo())
#Map.addLayer(bb,{},"Bounding box")

# get N, S, E, W coordinates from bounding box
# from https://gis.stackexchange.com/questions/318959/get-lon-lat-of-a-top-left-corner-for-geometry-in-google-earth-engine

## return the list of coordinates
listCoords = ee.Array.cat(bb.coordinates(), 1); 
##Casting it to an array makes it possible to slice out the x and y coordinates:
##get the X-coordinates
xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());

## Reducing each array reveals then the min and max values:
## reduce the arrays to find the max (or min) value
West = ee.Number(xCoords.reduce('min', [0]).get([0,0])).getInfo(); #print('West',West);
East = ee.Number(xCoords.reduce('max', [0]).get([0,0])).getInfo(); #print('East',East);
North = ee.Number(yCoords.reduce('min', [0]).get([0,0])).getInfo(); #print('North',North);
South = ee.Number(yCoords.reduce('max', [0]).get([0,0])).getInfo(); #print('South',South);

In [ ]:
# use bounding box to get geodataframe of all OSM data on recreation sites/parks. Use .drop to remove properties that have incompatible names with FeatureCollections

tags = {'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} 
RecSites = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
RecSites = RecSites.reset_index()
print(RecSites.shape[0])
print(RecSites.crs)
RecSites.sort_index()#.sample(3)
#RecSites.sort_values(by=['osmid'],axis='index')

In [ ]:
## keep only columns desired to reduce file size 
RecSites = RecSites[RecSites['element_type']!= 'node']
RecSites = RecSites[RecSites.geom_type != 'LineString']
RecSites = RecSites.loc[:, ['osmid','geometry']] #['element_type','osmid','leisure','boundary','name','geometry']
print(RecSites.shape[0])
RecSites.sort_index()#.sample(3)

In [ ]:
# convert Geodataframe to GeoJson
RecSitesGJ = geemap.gdf_to_geojson(RecSites) #RecSitesSimp
# convert GeoJson to ee.FeatureCollection
RecSitesFC = geemap.geojson_to_ee(RecSitesGJ)#.filter(ee.Filter.eq('element_type','way')).select(['osmid','element_type','geometry','leisure','boundary'])

In [ ]:
# add recreation sites to map 
Map.addLayer(RecSitesFC, {}, 'Recreation Sites')

In [ ]:
DistanceThres = 400

def amenityBuffer(feat):
  feat = ee.Feature(feat)
  return feat.buffer(DistanceThres)
RecCatchment = RecSitesFC.map(amenityBuffer)

RecCatchmentUnion = RecCatchment.union()
Map.addLayer(RecCatchmentUnion, {}, 'Recreation catchment')


In [ ]:
mask = ee.Image.constant(1).clip(RecCatchmentUnion.geometry()).mask()

popwAccess = popImg.updateMask(mask).rename('populationwOpenSpace')
#Map.addLayer(popwAccess, popviz, 'Population with open space access')

#Map

In [ ]:
## add tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')

Map2 = geemap.Map(height="350px")
Map2.add_basemap('HYBRID')
Map2.centerObject(Districts, zoom=10)

Map2.addLayer(popImg,popviz,"Population",False)
TreePctThreshold = 10 #whole numbers - 0-100
Map2.addLayer(TreeCover.updateMask(TreeCover.gte(TreePctThreshold)),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)

In [ ]:
# calcs for % population with threshold level (10%+?) of tree cover within walking distance (400m)

circle400m = ee.Kernel.circle(400, 'meters', False)
TreeCoverin400m = TreeCover.reduceNeighborhood(ee.Reducer.mean(), circle400m)

popwthresTC = popImg.updateMask(TreeCoverin400m.gte(10)).rename('populationwTreeCover')

#combImg = combImg.addBands([TreeCoverin400m])

Map2.addLayer(TreeCoverin400m,{'min':0, 'max':100},'TreeCoverin400m',False)
Map2.addLayer(popwthresTC, popviz, 'Population with access to tree cover')
Map2

In [ ]:
combImg = popImg.addBands([popwAccess,popwthresTC])

PopbyDistrict=combImg.reduceRegions(
  reducer= ee.Reducer.sum(), 
  collection= Districts, 
  scale= popScale, 
  tileScale= 1
)

#print(PopbyDistrict.first().toDictionary().getInfo())

In [ ]:
def calcs(feat):
     return feat.set({
        'PopwOpenSpaceAccessPct': feat.getNumber('populationwOpenSpace').divide(feat.getNumber('population')),
        'PopwTreeCoverAccessPct': feat.getNumber('populationwTreeCover').divide(feat.getNumber('population'))
     })
         
PopbyDistrict = PopbyDistrict.map(calcs).sort('PopwTreeCoverAccessPct',False)
#print(PopbyDistrict.first().toDictionary().getInfo())

In [ ]:
## display features in chart

import geemap.chart as chart

xProperty = 'shapeName' #'geo_name' #,"Woreda"
yProperties = ['PopwTreeCoverAccessPct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "% population with at least 10% tree cover within 5 minute walking distance",
    "legend_location": "top-right",
    "height": "500px",
}

chart.feature_byFeature(PopbyDistrict, xProperty, yProperties, **options)

In [ ]:
# use bounding box to get geodataframe of all OSM data on roads. Use .drop to remove properties that have incompatible names with FeatureCollections

tags = {'highway':['motorway','trunk','primary','secondary','tertiary','residential','unclassified','road']} #{'highway':True} 
Roads = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
Roads = Roads.reset_index()
print(Roads.shape[0])
print(Roads.crs)
#Roads.sort_index()#.sample(3)

In [ ]:
Roads = Roads.loc[:, ['osmid','geometry']]
RoadsGJ = geemap.gdf_to_geojson(Roads) #RecSitesSimp
RoadsFC = geemap.geojson_to_ee(RoadsGJ)
#print(RecSitesFC.first().toDictionary().getInfo())


In [ ]:
Map.addLayer(RoadsFC, {}, 'Roads')
Map

In [ ]:
## Rasterize OSM roads.
def numeric(feature):
    return feature.set({Numeric: 1})
RoadsR = RoadsFC.map(numeric)

OSMroadsImg = RoadsR.reduceToImage({
    'properties': ['Numeric'],
    'reducer': ee.Reducer.first()
  });

In [ ]:
## Make an image, with the same projection as WorldCover, out of the OSM ways in the FC.

WCprojection = WorldCover.projection(); 
#RecSitesImg = RecSitesFC.filter(ee.Filter.eq('element_type','way')).reduceToImage(properties=['osmid'],reducer=ee.Reducer.first());

RecSitesImg = RecSitesFC.style(
  color='gray',
).reproject(
      crs= WCprojection
    )
#print(RecSitesImg.getInfo())

## Display image.
#Map.addLayer(RecSitesImg, {},"Rec Sites Image");

In [ ]:
# create image with two bands: BuiltupPixels and UrbanOpenPixels
Builtup = WorldCover.updateMask(WorldCover.eq(50)).rename("BuiltupPixels")
UrbanOpen = RecSitesImg.updateMask(WorldCover.eq(50)).select(1).rename("UrbanOpenPixels")

#Map.addLayer(Builtup, {},"Builtup");
#Map.addLayer(UrbanOpen, {},"UrbanOpen");

comb = Builtup.addBands([UrbanOpen])
#print(comb.getInfo)

In [ ]:
## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature

OpenBuiltcount=comb.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

#print(OpenBuiltcount.limit(1).getInfo())

In [ ]:
# Function to translate pixel counts into area and percents

def CountToArea(feat):
    feat = ee.Feature(feat)
    FeatArea = ee.Number(feat.area(1)).multiply(0.000001)
    UrbanOpenArea = ee.Number(feat.getNumber('UrbanOpenPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    BuiltupArea = ee.Number(feat.getNumber('BuiltupPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    OpenAreaPctofBuiltUpArea = ee.Number(UrbanOpenArea).divide(ee.Number(BuiltupArea))
    
    cityID = Areaofinterest
    #geo_level = unitofanalysis
    geo_level = feat.getString("shapeID").split('-').getString(1)
    #geo_name = feat.getString("Sub_City").cat(ee.String("-")).cat(feat.getString("Woreda"))
    #geo_name = feat.getString("city_name_viz").split(' ').join('_')
    geo_name = feat.getString("shapeName").split(' ').join('_')
    geo_id = ee.String(cityID+"-").cat(geo_name)
    year = 2020
    source = "OpenStreetMap, ESA WorldCover"

    return feat.set({
        'TotalareaKM2': FeatArea,
        'UrbanOpenAreaKM2': UrbanOpenArea,
        'BuiltupAreaKM2': BuiltupArea,
        'OpenAreaPctofBuiltUpArea': OpenAreaPctofBuiltUpArea,
        'geo_level': geo_level,
        'geo_name': geo_name,
        'geo_id': geo_id,
        'year':year,
        'source':source,
    })

# apply function to FeatureCollection
OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea)
print(OpenBuiltAreaPct.first().toDictionary().getInfo())

In [ ]:
Tpctfills = ee.Image().paint(**{'featureCollection': OpenBuiltAreaPct,'color': 'OpenAreaPctofBuiltUpArea'})

fillspalette = ['red', 'green']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':0.1}, 'Open Area as % of built-up area', True, 0.65)
Map

In [ ]:
OpenBuiltAreaPct = OpenBuiltAreaPct.sort('OpenAreaPctofBuiltUpArea', False)

In [ ]:
# Download OpenBuiltAreaPct as shapefile through URL

# function for converting GeometryCollection to Polygon/MultiPolygon
def filter_polygons(ftr):
    geometries = ftr.geometry().geometries()
    geometries = geometries.map(
        lambda geo: ee.Feature(ee.Geometry(geo)).set('geoType', ee.Geometry(geo).type())
    )

    polygons = (
        ee.FeatureCollection(geometries)
        .filter(ee.Filter.eq('geoType', 'Polygon'))
        .geometry()
    )
    return ee.Feature(polygons).copyProperties(ftr)

OpenBuiltAreaPctSHP = OpenBuiltAreaPct.map(filter_polygons)

col_names = OpenBuiltAreaPctSHP.first().propertyNames().getInfo()
#print("Column names: ", col_names)

url = OpenBuiltAreaPctSHP.getDownloadURL("shp", col_names, 'OpenBuiltAreaPctSHP')
#print(url)


In [ ]:
# Download FeatureCollection of OpenBuiltAreaPct as shapefile to Google Drive

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'SHP',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}
#print('Exporting {}'.format(OpenBuiltAreaPct))
#task = ee.batch.Export.table.toDrive(OpenBuiltAreaPct, 'OpenBuiltAreaPct', **task_config)
#task.start()

In [ ]:

# Download FeatureCollection of OSM data (ways/polygons only) to Google Drive as geojson
# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'Geojson',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}

#print('Exporting {}'.format(RecSitesFC))
#task = ee.batch.Export.table.toDrive(RecSitesFC, 'RecSites', **task_config)
#task.start()